In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gensim
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import spacy
import sklearn as sk

Import our topic modeling library `BERTopic` and openai modules

In [ ]:
%pip install git+https://github.com/MaartenGr/BERTopic huggingface_hub safetensors -qqq

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 73, in resolve
    collected = self.factory.collect_root_requirements(root_reqs)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 491, in collect_root_requirements
    req = self._make_requirement_from_install_req(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 453, in _make_requirement_fro

In [ ]:
import bertopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction

Load dataset

In [ ]:
dir = '/content/drive/MyDrive/15kdataset.csv'

# load data

In [ ]:


train_dfs = [ pd.read_csv( f'{dir}', encoding='utf8' ) ]
print( train_dfs[ 0 ].shape )
train_dfs[ 0 ].head()

(15281, 8)


,Title,DOI,Author,Full_Text_Url,Question,Answer,Topic_name,Topic_Words
0,NaN,NaN,NaN,NaN,What was the main aim of PbR?,The main aim of PbR was to improve efficiency ...,NaN,NaN
1,NaN,NaN,NaN,NaN,When was the first NHS foundation trust applic...,The first NHS foundation trust applicants move...,NaN,NaN
2,NaN,NaN,NaN,NaN,How often are HRGs reviewed?,HRGs are reviewed every four years.,NaN,NaN
3,NaN,NaN,NaN,NaN,What is the aim of the Payment by Results system?,The goal of Payment by Results is to allow the...,NaN,NaN
4,NaN,NaN,NaN,NaN,What was the purpose of introducing the Commis...,The CQUIN payment framework was introduced nat...,NaN,NaN


In [ ]:
train_dfs = pd.read_csv(dir, encoding='utf8')
print(train_dfs.shape)
train_dfs.head()

(15281, 8)


,Title,DOI,Author,Full_Text_Url,Question,Answer,Topic_name,Topic_Words
0,NaN,NaN,NaN,NaN,What was the main aim of PbR?,The main aim of PbR was to improve efficiency ...,NaN,NaN
1,NaN,NaN,NaN,NaN,When was the first NHS foundation trust applic...,The first NHS foundation trust applicants move...,NaN,NaN
2,NaN,NaN,NaN,NaN,How often are HRGs reviewed?,HRGs are reviewed every four years.,NaN,NaN
3,NaN,NaN,NaN,NaN,What is the aim of the Payment by Results system?,The goal of Payment by Results is to allow the...,NaN,NaN
4,NaN,NaN,NaN,NaN,What was the purpose of introducing the Commis...,The CQUIN payment framework was introduced nat...,NaN,NaN


# preprocess (without lemmatization)

The only steps involved here are to concatentate `Question` and `Answer` and remove footnote numbers

In [ ]:
import pandas as pd

# Initialize the train_dfs list
train_dfs = []

# Populate the train_dfs list with DataFrames
train_dfs.append(pd.read_csv(dir))
# Add more files as needed

# Loop through the train_dfs list
for df in train_dfs:
    df['text'] = df.Question + ' ' + df.Answer.str.replace(r'\[[0-9]*\]', '')
    df['text'] = df['text'].astype(str)  # Convert float to string


with remove stopwords, speech tagging


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Initialize stopwords
stop_words = set(stopwords.words('english'))

# Initialize the train_dfs list
train_dfs = []

# Populate the train_dfs list with DataFrames
train_dfs.append(pd.read_csv(dir))
# Add more files as needed

# Loop through the train_dfs list
for df in train_dfs:
    df['text'] = df.Question + ' ' + df.Answer.str.replace(r'\[[0-9]*\]', '')
    df['text'] = df['text'].astype(str)  # Convert float to string

    # Tokenize words and remove stopwords
    df['tokenized_text'] = df['text'].apply(lambda x: word_tokenize(x))
    df['filtered_text'] = df['tokenized_text'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

    # Tag parts of speech
    df['pos_tags'] = df['filtered_text'].apply(pos_tag)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
train_dfs[0].head()

,Title,DOI,Author,Full_Text_Url,Question,Answer,Topic_name,Topic_Words,text,tokenized_text,filtered_text,pos_tags
0,NaN,NaN,NaN,NaN,What was the main aim of PbR?,The main aim of PbR was to improve efficiency ...,NaN,NaN,What was the main aim of PbR? The main aim of ...,"[What, was, the, main, aim, of, PbR, ?, The, m...","[main, aim, PbR, ?, main, aim, PbR, improve, e...","[(main, JJ), (aim, NN), (PbR, NNP), (?, .), (m..."
1,NaN,NaN,NaN,NaN,When was the first NHS foundation trust applic...,The first NHS foundation trust applicants move...,NaN,NaN,When was the first NHS foundation trust applic...,"[When, was, the, first, NHS, foundation, trust...","[first, NHS, foundation, trust, applicants, mo...","[(first, RB), (NHS, NNP), (foundation, NN), (t..."
2,NaN,NaN,NaN,NaN,How often are HRGs reviewed?,HRGs are reviewed every four years.,NaN,NaN,How often are HRGs reviewed? HRGs are reviewed...,"[How, often, are, HRGs, reviewed, ?, HRGs, are...","[often, HRGs, reviewed, ?, HRGs, reviewed, eve...","[(often, RB), (HRGs, NNP), (reviewed, VBD), (?..."
3,NaN,NaN,NaN,NaN,What is the aim of the Payment by Results system?,The goal of Payment by Results is to allow the...,NaN,NaN,What is the aim of the Payment by Results syst...,"[What, is, the, aim, of, the, Payment, by, Res...","[aim, Payment, Results, system, ?, goal, Payme...","[(aim, JJ), (Payment, NNP), (Results, NNP), (s..."
4,NaN,NaN,NaN,NaN,What was the purpose of introducing the Commis...,The CQUIN payment framework was introduced nat...,NaN,NaN,What was the purpose of introducing the Commis...,"[What, was, the, purpose, of, introducing, the...","[purpose, introducing, Commissioning, Quality,...","[(purpose, JJ), (introducing, VBG), (Commissio..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine all texts into one list
all_texts = []
for df in train_dfs:
    all_texts.extend(df['filtered_text'])

# Convert list of tokens to list of strings
all_texts = [' '.join(tokens) for tokens in all_texts]

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed

# Fit and transform the texts
tfidf_matrix = tfidf_vectorizer.fit_transform(all_texts)

# Get the feature names (words)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Get the top words per document
top_words_per_doc = []
for i, text in enumerate(all_texts):
    feature_index = tfidf_matrix[i,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
    sorted_tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)
    top_words = [feature_names[i] for i, _ in sorted_tfidf_scores][:10]  # Adjust the number of top words as needed
    top_words_per_doc.append(top_words)

# Print top words for each document
for i, top_words in enumerate(top_words_per_doc):
    print(f"Document {i+1}: {', '.join(top_words)}")


Streaming output truncated to the last 5000 lines.
Document 10282: storage, thermal, medium, suitable, make, energy, properties, temperatures, capacity, economic
Document 10283: roof, pcm, mm, used, cm, panel, materials, 15
Document 10284: roof, pcm, panel
Document 10285: range, temperature, used
Document 10286: roof, using
Document 10287: southern, city, al, climate, classification, according, type
Document 10288: southern, city, season, summer, weather, al, like, day, temperatures, respectively
Document 10289: city, summer, 2021, al, climate, east, like, highest, temperatures, high
Document 10290: pcm, trend, indicate, since, elements, positive, impact, temperature
Document 10291: temperature
Document 10292: thermal
Document 10293: pcm, elements, reference, lag, showed, difference, compared, time, temperature
Document 10294: temperature, fig, west, east, radiation, indoor, south, north, surface, solar
Document 10295: roof, reference, surface, maximum, temperature, second, cycle, firs

Print out top words of 10 each for each column


In [ ]:
with open('/content/drive/MyDrive/top_words_per_doc.csv', 'w') as f:
    for i, top_words in enumerate(top_words_per_doc):
        f.write(f"Document {i+1}: {', '.join(top_words)}\n")
